In [ ]:
#here we requests the dataset from googleapis.com 
#should return a zip file 
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip


--2020-12-12 22:13:51--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.196.128, 108.177.112.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.196.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   154MB/s    in 0.4s    

2020-12-12 22:13:51 (154 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import os 
import zipfile
#unzip  the zip file
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(150, 150, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

# Let's create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
keras_model = Model(img_input, output)

#Let's compile
from tensorflow.keras.optimizers import RMSprop

keras_model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])


#Let's generate image data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')


#let's train!
keras_history = keras_model.fit_generator(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
100/100 - 56s - loss: 0.8066 - acc: 0.5620 - val_loss: 0.6729 - val_acc: 0.6230
Epoch 2/15
100/100 - 56s - loss: 0.6463 - acc: 0.6470 - val_loss: 0.6698 - val_acc: 0.5850
Epoch 3/15
100/100 - 56s - loss: 0.5705 - acc: 0.7025 - val_loss: 0.6055 - val_acc: 0.6850
Epoch 4/15
100/100 - 56s - loss: 0.4901 - acc: 0.7600 - val_loss: 0.6176 - val_acc: 0.6780
Epoch 5/15
100/100 - 56s - loss: 0.4209 - acc: 0.8000 - val_loss: 0.6266 - val_acc: 0.7060
Epoch 6/15
100/100 - 56s - loss: 0.3314 - acc: 0.8650 - val_loss: 0.6696 - val_acc: 0.6980
Epoch 7/15
100/100 - 56s - loss: 0.2788 - acc: 0.8880 - val_loss: 0.6392 - val_acc: 0.6990
Epoch 8/15
100/100 - 56s - loss: 0.1812 - acc: 0.9275 - val_loss: 0.7027 - val_acc: 0.7020
Epoch 9/15
100/100 - 56s - loss: 0.1203 - acc: 0.9495 - val_loss: 1.1166 - val_acc: 0.6980
Epoch 10/15
100/100 -

In [ ]:
keras_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)       

In [ ]:
pip install onnxruntime

     |████████████████████████████████| 4.1MB 6.6MB/s 


In [ ]:
pip install keras2onnx

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import keras2onnx
import onnxruntime

onnx_model = keras2onnx.convert_keras(keras_model, keras_model.name)

# runtime prediction
#content = onnx_model.SerializeToString()

#sess = onnxruntime.InferenceSession(content)
#x = x if isinstance(x, list) else [x]
#feed = dict([(input.name, x[n]) for n, input in enumerate(sess.get_inputs())])
#pred_onnx = sess.run(None, feed)
temp_model_file = 'model.onnx'
keras2onnx.save_model(onnx_model, temp_model_file)
sess = onnxruntime.InferenceSession(temp_model_file)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 35 -> 18


In [ ]:
ls

model.onnx  sample_data/
